<a href="https://colab.research.google.com/github/kevinkhu/ScalingK2/blob/main/LAMOST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Work with [LAMOST Survey](http://dr7.lamost.org/) Data

---

Import important things.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from astropy.table import Table
import warnings
warnings.filterwarnings('ignore')

Download and read in the **LAMOST AFGK** stellar catalog as a Pandas DataFrame (this will take 1-2 minutes, we'll work with M stars below).

In [2]:
df = pd.read_csv('http://dr7.lamost.org/v2.0/catdl?name=dr7_v2.0_LRS_stellar.csv.gz', compression='gzip', header=0, sep='|', dtype={'gaia_source_id': object})

Print a list of all the fields in the table. A description of the columns can be found [here](http://dr7.lamost.org/v2.0/doc/lr-data-production-description#S3.2).

In [3]:
df.columns

Index(['obsid', 'designation', 'obsdate', 'lmjd', 'mjd', 'planid', 'spid',
       'fiberid', 'ra_obs', 'dec_obs', 'snru', 'snrg', 'snrr', 'snri', 'snrz',
       'objtype', 'class', 'subclass', 'z', 'z_err', 'magtype', 'mag1', 'mag2',
       'mag3', 'mag4', 'mag5', 'mag6', 'mag7', 'tsource', 'fibertype', 'tfrom',
       'tcomment', 'offsets', 'offsets_v', 'ra', 'dec', 'teff', 'teff_err',
       'logg', 'logg_err', 'feh', 'feh_err', 'rv', 'rv_err', 'alpha_m',
       'alpha_m_err', 'gaia_source_id', 'gaia_g_mean_mag'],
      dtype='object')

Set a signal-to-noise threshold, for example 'snrg' > 20.

In [4]:
dfq = df[df['snrg']>20]
dfq.reset_index(inplace=True,drop=True)
dfq

,obsid,designation,obsdate,lmjd,mjd,planid,spid,fiberid,ra_obs,dec_obs,snru,snrg,snrr,snri,snrz,objtype,class,subclass,z,z_err,magtype,mag1,mag2,mag3,mag4,mag5,mag6,mag7,tsource,fibertype,tfrom,tcomment,offsets,offsets_v,ra,dec,teff,teff_err,logg,logg_err,feh,feh_err,rv,rv_err,alpha_m,alpha_m_err,gaia_source_id,gaia_g_mean_mag
0,101008,J220928.49-015720.7,2011-10-24,55859,55858,F5902,1,8,332.368745,-1.955771,1.21,20.64,37.99,48.37,27.99,Star,STAR,F9,0.000087,0.000032,ugriz,18.25,16.64,15.97,15.77,15.64,99.0,99.0,JF_LEGAS_S,Obj,-,.,0,0.0,332.368745,-1.955771,5580.10,110.21,4.594,0.177,-0.019,0.116,26.20,9.45,0.191834,0.046798,2676865000324707584,16.039270
1,101016,J220923.69-020809.9,2011-10-24,55859,55858,F5902,1,16,332.348725,-2.136096,1.23,31.76,66.01,91.37,60.55,Star,STAR,K3,-0.000044,0.000020,ugriz,18.64,16.21,15.23,14.85,14.62,99.0,99.0,JF_LEGAS_S,Obj,-,.,0,0.0,332.348725,-2.136096,4588.43,52.84,4.691,0.074,-0.228,0.052,-13.11,6.14,-9999.000000,-9999.000000,2676107368093944576,15.277764
2,101017,J220946.66-015526.5,2011-10-24,55859,55858,F5902,1,17,332.444417,-1.924046,2.34,24.14,40.95,52.28,30.69,Star,STAR,G0,0.000006,0.000029,ugriz,17.97,16.53,16.00,15.78,15.65,99.0,99.0,JF_LEGAS_S,Obj,-,.,0,0.0,332.444417,-1.924046,5719.72,90.40,3.966,0.141,-0.235,0.094,1.73,8.56,0.154328,0.050794,2676120459154270336,16.003653
3,101020,J220853.37-015915.4,2011-10-24,55859,55858,F5902,1,20,332.222379,-1.987626,2.30,23.38,38.51,51.88,29.74,Star,STAR,F6,0.000052,0.000030,ugriz,17.01,15.98,15.51,15.35,15.27,99.0,99.0,JF_LEGAS_S,Obj,-,.,0,0.0,332.222379,-1.987626,5931.69,95.66,4.371,0.150,-1.029,0.100,15.56,8.87,0.189192,0.051621,2676863969532562688,15.540595
4,101021,J220924.33-014833.5,2011-10-24,55859,55858,F5902,1,21,332.351381,-1.809333,6.74,42.75,66.50,77.70,48.28,Star,STAR,F5,-0.000082,0.000018,ugriz,16.75,15.61,15.16,14.98,14.92,99.0,99.0,JF_LEGAS_S,Obj,-,.,0,0.0,332.351381,-1.809333,6080.15,39.89,4.145,0.057,-1.025,0.036,-24.65,5.50,0.136758,0.031056,2676876029800730368,15.178959
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4550157,746915239,J200744.40+192327.5,2019-06-08,58643,58642,HD200934N184657B01,15,239,301.935019,19.390986,1.17,22.52,58.67,85.21,64.48,Star,STAR,G5,0.000059,0.000019,gri,16.12,15.25,14.77,99.00,99.00,99.0,99.0,LEGUE_LCH,Obj,LCH100002,14461990929302608,0,0.0,301.935019,19.390986,4736.02,62.38,3.259,0.099,-0.076,0.065,17.82,5.64,0.331381,0.057650,1822593668710339328,15.217259
4550158,746915240,J200641.48+192721.5,2019-06-08,58643,58642,HD200934N184657B01,15,240,301.672867,19.455989,7.86,58.02,104.74,137.68,102.88,Star,STAR,G8,-0.000029,0.000016,gri,15.20,14.48,14.18,99.00,99.00,99.0,99.0,LEGUE_LCH,Obj,LCH100002,14462092237881009,0,0.0,301.672867,19.455989,5062.01,35.28,3.351,0.050,-0.315,0.030,-8.68,4.73,0.120561,0.037522,1822608821355250560,14.511239
4550159,746915246,J200709.18+193459.0,2019-06-08,58643,58642,HD200934N184657B01,15,246,301.788287,19.583069,5.37,56.06,108.50,145.17,109.70,Star,STAR,G5,-0.000190,0.000017,gri,15.34,14.50,14.15,99.00,99.00,99.0,99.0,LEGUE_LCH,Obj,LCH100002,14461968519991892,0,0.0,301.788287,19.583069,4819.22,38.97,3.314,0.056,-0.017,0.034,-56.99,5.23,-9999.000000,-9999.000000,1822611776292724480,14.542370
4550160,746915248,J200657.84+192831.0,2019-06-08,58643,58642,HD200934N184657B01,15,248,301.741039,19.475298,17.54,81.56,127.34,160.08,114.79,Star,STAR,G2,-0.000144,0.000016,gri,14.62,14.17,13.96,99.00,99.00,99.0,99.0,LEGUE_LCH,Obj,LCH100003,14461966817725687,0,0.0,301.741039,19.475298,5868.43,34.44,3.999,0.048,0.132,0.029,-43.20,4.66,-9999.000000,-9999.000000,1822608546477297536,14.180772


Create a smaller DataFrame with only the data you plan to work with.

In [5]:
lamostafgk = dfq[['obsid','gaia_source_id','subclass','teff','teff_err','logg','logg_err','feh','feh_err']].copy()
lamostafgk.rename(columns={'teff_err':'e_teff','logg_err':'e_logg','feh_err':'e_feh','gaia_source_id':'GaiaDR2ID'},inplace=True)
lamostafgk

,obsid,GaiaDR2ID,subclass,teff,e_teff,logg,e_logg,feh,e_feh
0,101008,2676865000324707584,F9,5580.10,110.21,4.594,0.177,-0.019,0.116
1,101016,2676107368093944576,K3,4588.43,52.84,4.691,0.074,-0.228,0.052
2,101017,2676120459154270336,G0,5719.72,90.40,3.966,0.141,-0.235,0.094
3,101020,2676863969532562688,F6,5931.69,95.66,4.371,0.150,-1.029,0.100
4,101021,2676876029800730368,F5,6080.15,39.89,4.145,0.057,-1.025,0.036
...,...,...,...,...,...,...,...,...,...
4550157,746915239,1822593668710339328,G5,4736.02,62.38,3.259,0.099,-0.076,0.065
4550158,746915240,1822608821355250560,G8,5062.01,35.28,3.351,0.050,-0.315,0.030
4550159,746915246,1822611776292724480,G5,4819.22,38.97,3.314,0.056,-0.017,0.034
4550160,746915248,1822608546477297536,G2,5868.43,34.44,3.999,0.048,0.132,0.029


**LAMOST M** Stars

Lather. Rinse. Repeat.

In [6]:
df = pd.read_csv('http://dr7.lamost.org/v2.0/catdl?name=dr7_v2.0_LRS_mstellar.csv.gz', compression='gzip', header=0, sep='|', dtype={'gaia_source_id': object})
print(df.columns)

#M stars are brighter in i band, so use that instead of g for snr cut.
dfq = df[df['snri']>20]
dfq.reset_index(inplace=True,drop=True)
lamostm = dfq[['obsid','gaia_source_id','subclass','teff','teff_err','logg','logg_err','m_h','m_h_err']].copy()
lamostm.rename(columns={'gaia_source_id':'GaiaDR2ID','teff_err':'e_teff','logg_err':'e_logg','m_h_err':'e_m_h'},inplace=True)
lamostm

Index(['obsid', 'designation', 'obsdate', 'lmjd', 'mjd', 'planid', 'spid',
       'fiberid', 'ra_obs', 'dec_obs', 'snru', 'snrg', 'snrr', 'snri', 'snrz',
       'objtype', 'class', 'subclass', 'z', 'z_err', 'magtype', 'mag1', 'mag2',
       'mag3', 'mag4', 'mag5', 'mag6', 'mag7', 'tsource', 'fibertype', 'tfrom',
       'tcomment', 'offsets', 'offsets_v', 'ra', 'dec', 'teff', 'teff_err',
       'logg', 'logg_err', 'm_h', 'm_h_err', 'ewha', 'ewha_err', 'tio5',
       'cah2', 'cah3', 'tio1', 'tio2', 'tio3', 'tio4', 'cah1', 'caoh',
       'tio5_err', 'cah2_err', 'cah3_err', 'tio1_err', 'tio2_err', 'tio3_err',
       'tio4_err', 'cah1_err', 'caoh_err', 'zeta', 'zeta_err', 'type', 'na',
       'gaia_source_id', 'gaia_g_mean_mag'],
      dtype='object')


,obsid,GaiaDR2ID,subclass,teff,e_teff,logg,e_logg,m_h,e_m_h
0,101050,2676910183380728576,dM0,3653.34,90.86,4.500,0.149,-1.000,0.108
1,101052,2677152728773788160,dM0,3957.58,82.04,5.056,0.136,-0.606,0.099
2,101090,2676891384309042560,dM0,3769.94,89.46,4.546,0.147,-1.000,0.108
3,101099,2676893961289434112,dM2,3378.83,93.14,4.809,0.153,-0.691,0.111
4,101109,2676878843004493568,dM1,3751.99,90.71,4.814,0.150,-0.373,0.108
...,...,...,...,...,...,...,...,...,...
531953,587101131,3302583186360081920,sdM4,3330.68,91.14,5.153,0.147,-2.153,0.107
531954,318912183,195638891365443968,sdM4,3300.00,74.46,4.832,0.120,-1.000,0.092
531955,368005093,314684561492375808,sdM4,3328.00,76.04,5.379,0.122,-1.000,0.093
531956,723702205,585166208631259648,sdM6,3203.30,86.14,4.984,0.142,-1.827,0.104
